# Analyze data using the Orbit API

This notebook shows how to analyze images using the Orbit API. A model has been generated and uploaded to the server.

### Description

The following section shows:
  * how to connect to OMERO from Orbit.
  * how to retrieve an Orbit model already saved in OMERO.
  * how to segment an Image.
  * how to save the generated Regions of Interest (ROI).

### Setup: Add Orbit to the Classpath

In [1]:
%classpath add jar /srv/conda/orbit/lib/*

## Enter credentials. Do not re-run this cell. 

In [2]:
g = new EasyForm("Enter credentials and continue to the next cell. Do not re-run this cell")
g.addTextField("Server").onInit({g['Server'] = "outreach.openmicroscopy.org"})
g.addTextField("UserName")
g.addPasswordField("Password")
g

## Collect the credentials and Connect to OMERO

In [24]:
import com.actelion.research.orbit.imageprovider.ImageProviderOmero
import com.actelion.research.orbit.imageprovider.OmeroConf
import com.actelion.research.orbit.imageAnalysis.dal.DALConfig

String hostName = g['Server']
String userName = g['UserName']
String password = g['Password']

imageProvider = new ImageProviderOmero(new OmeroConf(hostName, 4064, true))

imageProvider.authenticateUser(userName, password)

//Set the provider
DALConfig.setImageProvider(imageProvider)

println("Connected ")

Connected 


null

## Select the Image. Do not re-run this cell.

In [4]:
ga = new EasyForm("Select the Image to analyze")
ga.addTextField("ImageID")
ga

In [6]:
omeroImageId = ga['ImageID'].toLong()

10001

## Load the Image from the OMERO server

Load the image so we can get information like size.

In [7]:
import omero.gateway.Gateway
import omero.gateway.SecurityContext
import omero.gateway.facility.BrowseFacility

Gateway gateway = imageProvider.getGatewayAndCtx().getGateway()
SecurityContext ctx = imageProvider.getGatewayAndCtx().getCtx()

browse = gateway.getFacility(BrowseFacility)

image = browse.getImage(ctx, omeroImageId)

omero.gateway.model.ImageData (id=10001)

## Load the Orbit model

We use the most recent model added. Only the models owned by the currently logged in user are loaded.

In [21]:
import java.util.List
import com.actelion.research.orbit.beans.RawAnnotation
import com.actelion.research.orbit.imageAnalysis.models.OrbitModel

// Load Models that I own. OMERO annotations of type: Model
imageProvider.setOnlyOwnerObjects(true)
List<RawAnnotation> annotations = imageProvider.LoadRawAnnotationsByType(RawAnnotation.ANNOTATION_TYPE_MODEL)
println("Found " + annotations.size() + " files")
//Use the latest uploaded model
int fileAnnId = 0
for (RawAnnotation ra : annotations) {
    id = ra.getRawAnnotationId()
    if (id > fileAnnId) {
        fileAnnId = id
        break
    }
}
model = OrbitModel.LoadFromOrbit(fileAnnId)
println("Loaded Model: " + model.getName())

Found 2 files
Loaded Model: dundee-petr [svs]


null

## Segment the selected image.

We analyze a square around the center of the image.

In [22]:
import java.util.ArrayList
import java.util.List
import java.awt.Point

pixels = image.getDefaultPixels()
w = 1000
h = 1000
cx = (int) (pixels.getSizeX()/2)
cy = (int) (pixels.getSizeY()/2)
tiles = new ArrayList<Point>()
tiles.add(new Point(cx-w, cy-h))
tiles.add(new Point(cx+w, cy-h))
tiles.add(new Point(cx-w, cy+h))
tiles.add(new Point(cx+w, cy+h))

true

In [23]:
import com.actelion.research.orbit.imageAnalysis.models.SegmentationResult
import com.actelion.research.orbit.imageAnalysis.utils.OrbitHelper


// Perform the segmentation
result = OrbitHelper.Segmentation((int) omeroImageId, model, tiles, 1)

ending: svs [series 1]  format: SVS
type: 
ending: svs [series 1]  format: SVS
type: 
TaskResult:
Number of objects: 0


com.actelion.research.orbit.imageAnalysis.models.SegmentationResult@590701f2

## Save the ROIs back to OMERO

We retrieve the security context from the ``imageProvider``.

In [16]:
import java.util.ArrayList
import java.util.List
import omero.gateway.Gateway
import omero.gateway.SecurityContext

import java.awt.Shape

import omero.model.ImageI
import omero.model.PolygonI
import omero.model.RoiI

import static omero.rtypes.rstring
import static omero.rtypes.rint
import omero.gateway.facility.BrowseFacility
import omero.gateway.facility.DataManagerFacility

// Retrieve the security context
Gateway gateway = imageProvider.getGatewayAndCtx().getGateway()
SecurityContext ctx = imageProvider.getGatewayAndCtx().getCtx()

// Save the shapes
List<RoiI> roisToSave = new ArrayList<RoiI>()
for (Shape shape: result.shapeList) {
    // can cast shape to Polygon or simply listPoints
    String points = shape.listPoints()

    // Create Polygon in OMERO
    p = new PolygonI()
    // Convert "x, y; x, y" format to "x, y, x, y" for OMERO
    points = points.replace(";", ",")
    p.setPoints(rstring(points))
    p.setTheT(rint(0))
    p.setTheZ(rint(0))
    p.setStrokeColor(rint(-65281))   // yellow

    // Add each shape to an ROI on the Image
    ImageI image = new ImageI(omeroImageId, false)
    RoiI roi = new RoiI()
    roi.setImage(image)
    roi.addShape(p)
    roisToSave.add(roi)

}
gateway.getUpdateService(ctx).saveAndReturnArray(roisToSave)

[]


[]

## Close the connection

In [ ]:
imageProvider.close()

### License
Copyright (C) 2019-2020 University of Dundee. All Rights Reserved.
This program is free software; you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the
Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for
more details. You should have received a copy of the GNU General
Public License along with this program; if not, write to the
Free Software Foundation,
Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA. 